

## Fish Image Species Classification  

Given *images of fish*, let's try to predict the **species** of fish present in a given image.

We will use a TensorFlow/Keras pretrained CNN to make our predictions.

This dataset contains 9 different seafood types collected from a supermarket in Izmir, Turkey
for a university-industry collaboration project at Izmir University of Economics, and this work
was published in ASYU 2020.
The dataset includes gilt head bream, red sea bream, sea bass, red mullet, horse mackerel,
black sea sprat, striped red mullet, trout, shrimp image samples.

# Getting Started

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

from sklearn.model_selection import train_test_split

import tensorflow as tf

In [2]:
image_dir = Path('../input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset')

# Creating File DataFrame

In [3]:
# Get filepaths and labels
filepaths = list(image_dir.glob(r'**/*.png'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

# Concatenate filepaths and labels
image_df = pd.concat([filepaths, labels], axis=1)

# Drop GT images
image_df['Label'] = image_df['Label'].apply(lambda x: np.NaN if x[-2:] == 'GT' else x)
image_df = image_df.dropna(axis=0)

# Sample 200 images from each class
samples = []

for category in image_df['Label'].unique():
    category_slice = image_df.query("Label == @category")
    samples.append(category_slice.sample(200, random_state=1))

image_df = pd.concat(samples, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)

In [4]:
image_df

,Filepath,Label
0,../input/a-large-scale-fish-dataset/Fish_Datas...,Gilt-Head Bream
1,../input/a-large-scale-fish-dataset/Fish_Datas...,Sea Bass
2,../input/a-large-scale-fish-dataset/Fish_Datas...,Red Mullet
3,../input/a-large-scale-fish-dataset/Fish_Datas...,Shrimp
4,../input/a-large-scale-fish-dataset/Fish_Datas...,Trout
...,...,...
1795,../input/a-large-scale-fish-dataset/Fish_Datas...,Trout
1796,../input/a-large-scale-fish-dataset/Fish_Datas...,Red Sea Bream
1797,../input/a-large-scale-fish-dataset/Fish_Datas...,Striped Red Mullet
1798,../input/a-large-scale-fish-dataset/Fish_Datas...,Black Sea Sprat


In [5]:
train_df, test_df = train_test_split(image_df, train_size=0.7, shuffle=True, random_state=1)

# Loading the Images

In [6]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [7]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

Found 1008 validated image filenames belonging to 9 classes.
Found 252 validated image filenames belonging to 9 classes.
Found 540 validated image filenames belonging to 9 classes.


# Load Pretrained Model

In [8]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model.trainable = False

9412608/9406464 [==============================] - 0s 0us/step


# Training

In [9]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(9, activation='softmax')(x)


model = tf.keras.Model(inputs=inputs, outputs=outputs)


model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100
32/32 [==============================] - 49s 1s/step - loss: 1.3493 - accuracy: 0.5841 - val_loss: 0.2451 - val_accuracy: 0.9286
Epoch 2/100
32/32 [==============================] - 39s 1s/step - loss: 0.0845 - accuracy: 0.9789 - val_loss: 0.0941 - val_accuracy: 0.9722
Epoch 3/100
32/32 [==============================] - 38s 1s/step - loss: 0.0242 - accuracy: 0.9997 - val_loss: 0.0879 - val_accuracy: 0.9722
Epoch 4/100
32/32 [==============================] - 38s 1s/step - loss: 0.0109 - accuracy: 1.0000 - val_loss: 0.0817 - val_accuracy: 0.9722
Epoch 5/100
32/32 [==============================] - 38s 1s/step - loss: 0.0098 - accuracy: 0.9974 - val_loss: 0.0504 - val_accuracy: 0.9881
Epoch 6/100
32/32 [==============================] - 38s 1s/step - loss: 0.0048 - accuracy: 1.0000 - val_loss: 0.0533 - val_accuracy: 0.9841
Epoch 7/100
32/32 [==============================] - 39s 1s/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0449 - val_accuracy: 0.9881
Epoch 8/100
3

# Results

In [10]:
results = model.evaluate(test_images, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

    Test Loss: 0.02443
Test Accuracy: 99.44%
